In [18]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from gcn import ST_GCN
from ultralytics import YOLO
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = YOLO('82best.pt')
# 영상 분석
pred = model.predict('eating/res/23_result.mp4',save=True)

# temp = pred[:][:60]
# temp_tensor = temp[0].keypoints.data
# for t in temp[1:]:
#     temp_tensor = torch.cat((temp_tensor,t.keypoints.data),dim=0)
# temp_tensor = temp_tensor.permute(2,0,1)

# stgcn = ST_GCN(num_classes=13, 
#                   in_channels=3,
#                   t_kernel_size=5, #시간 그래프 convolution 커널 크기(t_kernel_size x 1)
#                   hop_size=1).to('cuda')
# stgcn.load_state_dict(torch.load('stgcns.pt'))

# result = torch.argmax(stgcn(temp_tensor.unsqueeze(0)),axis=1)


## time insert
# frame_add_time = cv2.putText(frame.plot(),f'{now.strftime("%H:%M:%S")}',(frame.plot().shape[0]//30,frame.plot().shape[1]//30),3,1,(0,0,0),1)
# time_list.append(now.strftime("%H:%M:%S"))
# frame_array.append(frame_add_time)



    WARNING ⚠️ stream/video/webcam/dir predict source will accumulate results in RAM unless `stream=True` is passed,
    causing potential out-of-memory errors for large sources or long-running streams/videos.

    Usage:
        results = model(source=..., stream=True)  # generator of Results objects
        for r in results:
            boxes = r.boxes  # Boxes object for bbox outputs
            masks = r.masks  # Masks object for segment masks outputs
            probs = r.probs  # Class probabilities for classification outputs

video 1/1 (1/210) /home/sungwook/yolo/eating/res/23_result.mp4: 640x384 1 dog, 126.5ms
video 1/1 (2/210) /home/sungwook/yolo/eating/res/23_result.mp4: 640x384 1 dog, 16.3ms
video 1/1 (3/210) /home/sungwook/yolo/eating/res/23_result.mp4: 640x384 1 dog, 16.3ms
video 1/1 (4/210) /home/sungwook/yolo/eating/res/23_result.mp4: 640x384 1 dog, 22.5ms
video 1/1 (5/210) /home/sungwook/yolo/eating/res/23_result.mp4: 640x384 1 dog, 16.4ms
video 1/1 (6/210) /home/sung

KeyboardInterrupt: 

In [38]:
# 거리 계산 코드
# 한 영상에 대해 action 빈도 수 측정 -> report
# 2초 (60 frame) 픽셀 이동거리 계산 * action 가중치

detect = False
ob_cnt = ob_not_cnt = 0
distance = 0
before = 0
frame = 0
for p in pred:
    frame+=1
    if frame%60==0:
        print(distance)
        distance = 0
    if p.boxes.conf >0.8: # conf 0.8 이상인 객체에 대해서
        if isinstance(before,torch.Tensor): # 이전 탐지 결과가 있으면
            distance+=(np.linalg.norm(before.cpu()-p.boxes.xywhn[0][:2].cpu())) # 거리 추가
            before = p.boxes.xywhn[0][:2]
        else: # 없으면 이전 탐지 결과로 생성
            before = p.boxes.xywhn[0][:2] 
    else: # conf가 0.8 이하인 객체거나 탐지가 안될 경우 탐지 결과 제거
        before = 0
print(distance)

In [5]:
import numpy as np
import torch
from stagcn import STA_GCN
from ultralytics import YOLO
import time
import cv2
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = YOLO('82best.pt')

### pooty_turn / noeat(walk) / body_sit_walk  -> weight 223    , cls 7 hop 2 att 2 kernel_size 3 -> walk
### eat                                       -> 204best_again , cls 8 hop 2 att 2 kernel_size 3

### demo0 - eat / noeat 
### demo1 - pooty_test ( pooty , turn )
### demo2 - body_sit_walk ( bodydown , sit , walk )
    
video_path = 'output.mp4'
eat_flag = False
if eat_flag:
    action_dict = {0: 'lying', 1: 'turn', 2: 'sit', 3: 'run', 4: 'walk', 5: 'bodydown', 6: 'potty', 7: 'eat'}
    stagcn_model = STA_GCN(num_classes=8, 
                  in_channels=2,
                  t_kernel_size=3,
                  hop_size=2,
                  num_att_edge=2).to(device)
    stagcn_model.load_state_dict(torch.load('weight_2.pt')) # cls = 8 , hop, att => 2 , 2 kernel = 3
else:
    action_dict = {0: 'lying', 1: 'turn', 2: 'sit',3: 'bodydown', 4: 'walk', 5: 'potty', 6: 'eat'}
    stagcn_model = STA_GCN(num_classes=7, 
                  in_channels=2,
                  t_kernel_size=3,
                  hop_size=2,
                  num_att_edge=2).to(device)
    stagcn_model.load_state_dict(torch.load('weight_1.pt')) # 219

count_dict = {v:0 for k,v in action_dict.items()}
stagcn_model.eval()

# YOLO 제너레이터로 프레임을 하나씩 처리
fps = int(cv2.VideoCapture(video_path).get(cv2.CAP_PROP_FPS))
pred_gen = model.predict(video_path, stream=True,verbose=False,save=True,conf=0.8)

frame_buffer = []
action_list = []  # 60 프레임을 저장하는 버퍼
con = []

fh =0
ft =0
time_list = []
frame_array = []

distance = 0
score = 0
before = 0
for i, frame in enumerate(pred_gen):
    now = time
    height = frame.orig_shape[0]
    width = frame.orig_shape[1]


    if i%2==0:
        if frame.boxes.conf.shape[0] and frame.boxes.conf[0]>0.8:
            if len(frame_buffer)==0:
                fh = i
            frame_buffer.append(frame)

    if i-fh>183: # 3간격 60프레임이 지나도록 차지 않았으면 ( 6초 연속 detection 되지 않았으면 ) clear
        frame_buffer.clear() 
        continue
    if len(frame_buffer) == 60:
        ft = i       
        temp_tensor = frame_buffer[0].keypoints.data
        before = frame_buffer[0][0].boxes.xywhn[0][:2] 
        for t in frame_buffer[1:]:
            temp_tensor = torch.cat((temp_tensor,temp_tensor[-1,:,:]*0.8+t[0].keypoints.data[:,:,:]*0.2),dim=0)
            distance+=(np.linalg.norm(before.cpu()-t.boxes.xywhn[0][:2].cpu())) # 거리 추가
            before = t.boxes.xywhn[0][:2]
        temp_tensor = temp_tensor.permute(2, 0, 1)[:2,:,:] # x , y 좌표만 활용
        temp_tensor[0] /= width
        temp_tensor[1] /= height
    
        with torch.no_grad():
            result = torch.argmax(stagcn_model(temp_tensor.unsqueeze(0))[1], axis=1)

            action = action_dict[result[0].item()]
            count_dict[action] +=1
            action_list.append(action)
            frame_buffer.clear()  # 버퍼 비우기
            print(f"Time :{fh//fps//60}분 {fh//fps%60}초 : {ft//fps//60}분 {ft//fps%60}초 Action : {action} , distance = {distance}")
            distance = 0

print("Count dict : " , count_dict)

<All keys matched successfully>

STA_GCN(
  (feature_extractor): FeatureExtractor(
    (bn): BatchNorm1d(30, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (stgc_block1): STGC_Block(
      (sgc): S_GC(
        (conv): Conv2d(2, 96, kernel_size=(1, 1), stride=(1, 1))
      )
      (tgc): Sequential(
        (0): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): ReLU()
        (2): Conv2d(32, 32, kernel_size=(3, 1), stride=(1, 1), padding=(1, 0))
        (3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (4): Dropout(p=0.5, inplace=False)
        (5): ReLU()
      )
      (residual): Sequential(
        (0): Conv2d(2, 32, kernel_size=(1, 1), stride=(1, 1))
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (stgc_block2): STGC_Block(
      (sgc): S_GC(
        (conv): Conv2d(32, 96, kernel_size=(1, 1), stride=(1, 1))
      )
      (tgc): Sequential(
        

Time :0분 0초 : 0분 4초 Action : walk , distance = 0.12070112896617502
Time :0분 5초 : 0분 9초 Action : walk , distance = 0.10239611618453637
Time :0분 10초 : 0분 14초 Action : sit , distance = 0.027391819341573864
Time :0분 15초 : 0분 19초 Action : sit , distance = 0.027755487011745572
Time :0분 20초 : 0분 25초 Action : sit , distance = 0.4779350811149925
Time :0분 25초 : 0분 30초 Action : sit , distance = 0.005077660083770752
Time :0분 30초 : 0분 35초 Action : bodydown , distance = 0.17519534344319254
Time :0분 35초 : 0분 40초 Action : bodydown , distance = 0.023787882993929088
Time :0분 40초 : 0분 45초 Action : bodydown , distance = 0.007827492780052125


Results saved to runs/pose/predict242


Count dict :  {'lying': 0, 'turn': 0, 'sit': 4, 'bodydown': 3, 'walk': 2, 'potty': 0, 'eat': 0}


In [23]:
fourcc = cv2.VideoWriter_fourcc(*'DIVX')
out = cv2.VideoWriter('time_eat.mp4',fourcc, 24, (width,height))

for i in range(len(frame_array)):
    out.write(frame_array[i])
out.release()

OpenCV: FFMPEG: tag 0x58564944/'DIVX' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


In [73]:
out = cv2.VideoWriter('/hi.mp4',-1, 30, (width,height))
for i in range(len(frame_array)):
    # writing to a image array
    print(i)
    out.write(frame_array[i])
out.release()

fourcc tag 0x7634706d/'mp4v' codec_id 000C
fourcc tag 0x31637661/'avc1' codec_id 001B
fourcc tag 0x33637661/'avc3' codec_id 001B
fourcc tag 0x31766568/'hev1' codec_id 00AD
fourcc tag 0x31637668/'hvc1' codec_id 00AD
fourcc tag 0x7634706d/'mp4v' codec_id 0002
fourcc tag 0x7634706d/'mp4v' codec_id 0001
fourcc tag 0x7634706d/'mp4v' codec_id 0007
fourcc tag 0x7634706d/'mp4v' codec_id 003D
fourcc tag 0x7634706d/'mp4v' codec_id 0058
fourcc tag 0x312d6376/'vc-1' codec_id 0046
fourcc tag 0x63617264/'drac' codec_id 0074
fourcc tag 0x7634706d/'mp4v' codec_id 00A3
fourcc tag 0x39307076/'vp09' codec_id 00A7
fourcc tag 0x31307661/'av01' codec_id 00E2
fourcc tag 0x6134706d/'mp4a' codec_id 15002
fourcc tag 0x63616c61/'alac' codec_id 15010
fourcc tag 0x6134706d/'mp4a' codec_id 1502D
fourcc tag 0x6134706d/'mp4a' codec_id 15001
fourcc tag 0x6134706d/'mp4a' codec_id 15000
fourcc tag 0x332d6361/'ac-3' codec_id 15003
fourcc tag 0x332d6365/'ec-3' codec_id 15028
fourcc tag 0x6134706d/'mp4a' codec_id 15004
fou

OpenCV: FFMPEG: format mp4 / MP4 (MPEG-4 Part 14)


In [ ]:
import cv2
import numpy as np
import sys

#합성시킬 두 개의 영상 열기
cap1 = cv2.VideoCapture('body_sit_walk_edit1.mp4')
cap2 = cv2.VideoCapture('body_sit_walk_edit2.mp4')

if not cap1.isOpened() or not cap2.isOpened():
	sys.exit()
    
#각 영상 프레임 수
frame_cnt1 = round(cap1.get(cv2.CAP_PROP_FRAME_COUNT))
frame_cnt2 = round(cap2.get(cv2.CAP_PROP_FRAME_COUNT))
fps = cap1.get(cv2.CAP_PROP_FPS)
effect_frames = int(fps*2)

delay = int(1000/fps)

#영상 가로 세로 설정
w = round(cap1.get(cv2.CAP_PROP_FRAME_WIDTH))
h = round(cap1.get(cv2.CAP_PROP_FRAME_HEIGHT))

#비디오 코덱 설정
fourcc = cv2.VideoWriter_fourcc(*'DIVX')

out = cv2.VideoWriter('output.avi',fourcc, fps, (w,h))

#1번 영상 열기
for i in range(frame_cnt1 - effect_frames):
    ret1, frame1 = cap1.read()
    if not ret1:
        break
        
    out.write(frame1)
    cv2.imshow('frame',frame1)
    cv2.waitKey(delay)
    
#합성하기
for i in range(effect_frames):
    ret1, frame1 = cap1.read()
    ret2, frame2 = cap2.read()
    
    #가중치 계산
    #만약 i가 1인경우 alpha는 1- 1/48 , 1-alpha는 0에 가까운 값
    alpha = 1.0 - i / effect_frames
    frame = cv2.addWeighted(frame1, alpha, frame2, 1-alpha,0)
    out.write(frame)
    
for i in range(effect_frames, frame_cnt2):
    ret2, frame2 = cap2.read()
    
    if not ret2:
        break
        
    out.write(frame2)
    cv2.imshow('frame', frame2)
    cv2.waitKey(delay)

In [ ]:
from stagcn import STA_GCN
from ultralytics import YOLO

def test(video_path, x3,y3,x4,y4):
    #video_path = '/home/jiwonnnnnnn/project/walk_data/walk_result/33_3_result.mp4'
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    action_dict = {0: 'lying', 1: 'turn', 2: 'sit', 3: 'run', 4: 'walk', 5: 'bodydown', 6: 'potty', 7: 'eat'}

    
    model = YOLO('/home/jiwonnnnnnn/project/walk_data/82best.pt')
    preds = model.predict(video_path,stream=True)
    frame_flag = 0

    stagcn_model = STA_GCN(num_classes=8, 
                  in_channels=2,
                  t_kernel_size=9,
                  hop_size=2,
                  num_att_edge=2).to(device)
    
    stagcn_model.load_state_dict(torch.load('/home/jiwonnnnnnn/project/stagcn_new.pt'))
    
    stagcn_model.eval()

    keypoints = []
    ious = []
    f_result = []

    with torch.no_grad():
        for i, pred in enumerate(preds):
            try:
                bbox = pred.boxes.data[0][:4]
                keypoint = pred.keypoints.data
                keypoints.append(keypoint)

                x1,y1,x2,y2 = bbox[0],bbox[1],bbox[2],bbox[3]

                iou = calculate_iou((x1,y1,x2,y2),(x3,y3,x4,y4))
                
                ious.append(iou)
                if iou > 0.65:
                    frame_flag += 1
                else:
                    frame_flag = 0

            except:
                print('none bbox')

            if i%60 == 0:
                k_tensor = torch.cat(keypoints, dim=0)
                print(k_tensor.shape)
                break
                k_tensor = k_tensor.permute(2,0,1)[:2,:,:] # [2, 1, 15]
                out = stagcn_model(k_tensor.unsqueeze(0))
                result = torch.argmax(out[1], axis=1)
                
                f_result.append(action_dict[result[0].item()])
                
                frame_flag = 0
                keypoints = []
    
    return f_result

In [ ]:
import numpy as np

def calculate_iou(box1, box2):
    
    x_min_inter = max(box1[0], box2[0])
    y_min_inter = max(box1[1], box2[1])
    x_max_inter = min(box1[2], box2[2])
    y_max_inter = min(box1[3], box2[3])

    intersection_area = max(0, x_max_inter - x_min_inter + 1) * max(0, y_max_inter - y_min_inter + 1)

    box1_area = (box1[2] - box1[0] + 1) * (box1[3] - box1[1] + 1)
    box2_area = (box2[2] - box2[0] + 1) * (box2[3] - box2[1] + 1)

    union_area = box1_area + box2_area - intersection_area

    iou = intersection_area / union_area

    return iou

In [35]:
phone_list = ['01024712421',
'01030099612',
'01027743612',
'01066368349',
'01075460075',
'01024712421',
'01021696805',
'01088726242',
'01088002761',
'01039282914',
'01089223398',
'01083093369',
'01027643730',
'01056308268',
'01049184161',
'01075584697',
'01074482652',
'01084746346',
'01029163634',
'01054471320',
'01030565580',
'01033286255',
'01062866923',
'01032880744',
'01052122421',
'01062348044',
'01063747665',
'01041856326',
'01090555465',
'01077696707',
'01023936900',
'01041636553',
'01071412928',
'01020670038',
'01087313070',
'01033149563',
'01047178609',
'01095523126',
'01071444301',
'01036604301', 
'01085625511',
'01071664301',
'01042578283',
'01062161091',
'01064100744',
'01091722090',
'01088132206',
'01062518091', 
'01022797938',
'01091153089',
'01095660628',
'01092034005',
'01034047166',
'01023331843',
'01027832963',
'01086842658',
'01048252369',
'01082999532',
'01071826254',
'01054234232',
'01055910479',
'01033042241',
'01054916594',
'01088002761',
'01041779495',
'01065090926',
'01037352976',
'01086147166',]

In [37]:
import random

cong= random.sample(phone_list,5)
cong

['01071664301', '01027743612', '01052122421', '01024712421', '01062161091']